In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.patches import Rectangle
from scipy.stats import poisson

import sys
sys.path.append('..')
sys.path.append('../Trot')
sys.path.append('../Data')

from Trot.Tsallis import TROT, q_log
from Trot.Generators import euc_costs  

# Influence of parameters $q$ and $\lambda$

We reproduce Figure 2. from (cite our paper)

First, choose values for $q$ and $\lambda$ to test,sample size n, modes $\mu$ and mixing coefficients $t$.

In [2]:
n = 100
q = np.arange(0.5,2,0.5)
l = [0.1,1,10,100]
mu1 = [20,60]
mu2 = [10,40,70]
t1 = [0.5,0.5]
t2 = [0.2,0.4,0.4]

Run TROT on selected parameters and marginals and produce the figure.

In [3]:
x = range(n)
    
r_tmp = []    
for mode in mu1:
    r_tmp.append(poisson.pmf(x,mode))
        
c_tmp = []    
for mode in mu2:
    c_tmp.append(poisson.pmf(x,mode))
        
r = np.dot(t1,r_tmp)
r = r/r.sum()
    
c = np.dot(t2,c_tmp)
c = c/c.sum()
    
    
M = euc_costs(n,n)
    
P = []
    
nq = len(q)
nl = len(l)
    
        
for j in range(nq):
    for i in range(nl):
        P_tmp = TROT(q[j],M,r,c,l[i],1E-2)
        P.append(P_tmp)
    
    
fig = plt.figure(figsize=(8, 8))
    
outer_grid = gridspec.GridSpec(2, 2, width_ratios=[1,5], height_ratios=[1,5])
outer_grid.update(wspace=0.01, hspace=0.01)
# gridspec inside gridspec
outer_joint = gridspec.GridSpecFromSubplotSpec(nq,nl, subplot_spec=outer_grid[1,1],wspace=0.02, hspace=0.02)
outer_row_marg = gridspec.GridSpecFromSubplotSpec(nq,1, subplot_spec=outer_grid[1,0],wspace=0.02, hspace=0.02)
outer_col_marg = gridspec.GridSpecFromSubplotSpec(1,nl, subplot_spec=outer_grid[0,1],wspace=0.02, hspace=0.02)
    
    
for b in range(nl):
    for a in range (nq):
        ax = plt.Subplot(fig, outer_joint[a,b])
        ax.imshow(P[nl*a + b], origin='upper', interpolation = None, aspect = 'auto', cmap = 'Greys')
        rect = Rectangle((0, 0), n-1, n-1, fc='none', ec='black')     
        rect.set_width(0.8)
        rect.set_bounds(0,0,n-1,n-1)
        ax.add_patch(rect)
        ax.set_xticks([])
        ax.set_yticks([])
        fig.add_subplot(ax)
        ax.set_axis_bgcolor('white')
        
for i in range(nq):
    ax_row = plt.Subplot(fig,outer_row_marg[i], sharey = ax)
    ax_row.plot(1-r, x)
    fig.add_subplot(ax_row)
    
    ax_row.axes.get_xaxis().set_visible(False)
    ax_row.axes.get_yaxis().set_visible(False)
    bottom, height = .25, .5
    top = bottom + height
    ax_row.text(-0.05, 0.5*(bottom+top), 'q = {0}'.format(q[i]), horizontalalignment='right', verticalalignment='center', rotation='vertical',transform=ax_row.transAxes, fontsize='medium')
    
    ax_row.set_axis_bgcolor('white')
    
for j in range(nl):
    ax_col = plt.Subplot(fig,outer_col_marg[j], sharex = ax)
    ax_col.plot(x,c)
    fig.add_subplot(ax_col)    
    bottom, height = .25, .5
    ax_col.axes.get_xaxis().set_visible(False)
    ax_col.axes.get_yaxis().set_visible(False)
    ax_col.set_title(r'$\lambda$'+' = {0}'.format(l[j]),fontsize='medium')
    ax_col.set_axis_bgcolor('white')
    
fig.show()

# Tsallis entropies

We plot the Tsallis entropies of a Bernoulli random variable with parameter $p$ for different values of $q$.

In [20]:
def Tsallis_Bernoulli(q,p):
    if np.isclose(q,1):
        return p*np.log(1/p) + (1-p)*np.log(1/(1-p))
    else:
        return p*q_log(q,1/p) + (1-p)*q_log(q,1/(1-p))

x = np.arange(1E-6,1,0.001)

plt.figure()

plt.plot(x,Tsallis_Bernoulli(0.1,x), label = 'q = 0.1')
plt.plot(x,Tsallis_Bernoulli(0.5,x), label = 'q = 0.5')
plt.plot(x,Tsallis_Bernoulli(1,x), label = 'q = 1')
plt.plot(x,Tsallis_Bernoulli(1.5,x), label = 'q = 1.5')
plt.plot(x,Tsallis_Bernoulli(5,x), label = 'q = 5')

plt.legend()

plt.show()

Notice how the choice of $q$ impacts the notion of randomness. When $q$ goes to 0, the entropy converges to the constant function $x=1$, except at the borders $p=0$ and $p=1$. Low values of q are anti-sparsity-inducing, but since they are rather flat around $p=0.5$ they do not require the distribution to be exactly uniform to have high-entropy. When $q$ goes to infinity, the entropy converges to the zero fonction.